In [0]:
""" here all the necessay modules and packages """

import numpy as np 
import pandas as pd 
import random
import sys
import cv2
import glob
import matplotlib.pyplot as plt200
from subprocess import check_output

import tensorflow
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Conv2D,MaxPooling2D,Input,Dense,Flatten,Dropout,Activation,BatchNormalization,GlobalAveragePooling2D
from tensorflow.python.keras.layers import PReLU
from tensorflow.python.keras import losses
from tensorflow.python.keras.optimizers import Adam, Adagrad
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras import regularizers
from sklearn.model_selection import GridSearchCV
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import keras
from tensorflow.python.keras.layers import LeakyReLU, PReLU
import os
from tensorflow.python.keras.applications.vgg16 import VGG16
# from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm
from tensorflow.python.keras.callbacks import TensorBoard


In [0]:
'''here now actuall process stats
first we used a keras image data generator'''

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    fill_mode="nearest",
    data_format='channels_last',
    brightness_range=[0.5, 1.5],
#     validation_split =0.2 
)


test_datagen = ImageDataGenerator(
#     horizontal_flip=True,
#                                   zoom_range=[0.5, 1.25],
    rescale=1./255, 
#     preprocessing_function=custom_func
 )


In [0]:
''' here i have used a flow_from_directory method from keras which requires 
    a folders with respective classes and images and reads that folders and automatically identifies the class.'''

train_generator = train_datagen.flow_from_directory(
    '/content/train_dir',
    target_size=(224,224),
    batch_size=32,
    color_mode = 'rgb',
    shuffle = True,
    seed = 42,
    class_mode='categorical',
    )



valid_generator = train_datagen.flow_from_directory(
    directory="/content/val_dir",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    )


# test_generator = test_datagen.flow_from_directory(
#     directory="/content/remake",
#     target_size=(512, 512),
#     color_mode="rgb",
#     batch_size=16,
#     class_mode=None,
#     shuffle=False,
#     seed=42
# )

In [0]:
train_generator.class_indices

In [0]:
'''transfer learning section'''

base_model=tensorflow.keras.applications.mobilenet.MobileNet()
base_model.summary()

In [0]:
x = base_model.layers[-6].output

# Create a new dense layer for predictions
# 7 corresponds to the number of classes
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

# inputs=mobile.input selects the input layer, outputs=predictions refers to the
# dense layer we created above.

model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [0]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
  return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
  return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [0]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

In [0]:
#class weight is for making prediction accurate. datasets is highly unbalanced.
class_weights={
    0: 1.0, # akiec
    1: 1.0, # bcc
    2: 1.0, # bkl
    3: 1.0, # df
    4: 3.0, # mel # Try to make the model more sensitive to Melanoma.
    5: 1.0, # nv
    6: 1.0, # vasc
}

In [0]:
#moddel checkpoint is for saving model in drive. Run this section only in colab
LOG_DIR = '/gdrive/My Drive/ML/cancer/training_logs/cancer'
MODEL_CHECKPOINT_NAME = 'model{epoch:03d}.hdf5'
MODEL_CHECKPOINT_DIRNAME = LOG_DIR + '/checkpoints/'

In [0]:
!mkdir -p "{MODEL_CHECKPOINT_DIRNAME}"

In [0]:
from tensorflow.python.keras.callbacks import TensorBoard
base_dir = './logs'
tbCallBack = TensorBoard(log_dir=base_dir,
                         write_graph=True,
                         write_grads=True,
                         batch_size=32,
                         write_images=True,
                         update_freq='batch')

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(base_dir)
)

In [0]:
get_ipython().system_raw('./ngrok authtoken 45Npt76HKnzZUgWh4ciwi_3wa9qRRs65bKe4SVbNLkz &')
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
import glob, os
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import load_model

In [0]:
def get_init_epoch(check_point_path):
    check_point_list = glob.glob(os.path.join(check_point_path, 'model*.hdf5'))
    base_names = [os.path.basename(check_point) for check_point in check_point_list]
    epochs = [int(re.search(r'\d+', string).group()) for string in base_names]
    return np.max(epochs) if epochs else 0

In [0]:
def load_saved_model(model_path):
    model = load_model(model_path)
    return model

In [0]:
import re

In [0]:
init_epoch = get_init_epoch(MODEL_CHECKPOINT_DIRNAME)
print(init_epoch)
if init_epoch:
    model_path = MODEL_CHECKPOINT_DIRNAME + MODEL_CHECKPOINT_NAME.format(epoch=init_epoch)
    model = load_saved_model(model_path)
    model.summary()
else:
    model =model

0


In [0]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [0]:

checkpoint = ModelCheckpoint(MODEL_CHECKPOINT_DIRNAME + MODEL_CHECKPOINT_NAME, monitor='val_top_3_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2, 
                              verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(train_generator, steps_per_epoch=1205, 
                              class_weight=class_weights,
                              validation_data=valid_generator,
                              validation_steps=29,
                              epochs=150, verbose=1,
                              callbacks=[model_checkpoint_callback, tbCallBack], initial_epoch=init_epoch)

In [0]:
# 143 -99 and 96 

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])


plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
import sys
import matplotlib
print("Generating plots...")
sys.stdout.flush()
matplotlib.use("Agg")
matplotlib.pyplot.style.use("ggplot")
matplotlib.pyplot.figure()
N =210
matplotlib.pyplot.plot(np.arange(0, N), history.history["loss"], label="train_loss")
matplotlib.pyplot.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
matplotlib.pyplot.plot(np.arange(0, N), history.history["acc"], label="train_acc")
matplotlib.pyplot.plot(np.arange(0, N), history.history["val_acc"], label="val_acc")
matplotlib.pyplot.title("Training Loss and Accuracy on diabetic retinopathy detection")
matplotlib.pyplot.xlabel("Epoch #")
matplotlib.pyplot.ylabel("Loss/Accuracy")
matplotlib.pyplot.legend(loc="lower left")
matplotlib.pyplot.savefig("plot.png")

In [0]:
from keras.models import load_model
from keras.models import Model
model=load_model=('/content/model.hdf5')

In [0]:
model.summary()

In [0]:

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

[0.2308176941393564, 0.92528737]

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
predictions